**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# Imports

In [ ]:
from itertools import product
import numpy as np
from scipy import optimize
from scipy import interpolate
import sympy as sm
import matplotlib.pyplot as plt
import pandas as pd 
from scipy import linalg
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize_scalar, root_scalar
import Exam as ex
import time
from mpl_toolkits.mplot3d import Axes3D
from Exam import problem2

%matplotlib inline

%load_ext autoreload
%autoreload 2

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $(1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

# Defining the parameters 

In [21]:
# Define the symbols

L = sm.symbols("L")
C = sm.symbols("C")
w = sm.symbols("w")
alpha = sm.symbols("alpha")
kappa = sm.symbols("kappa")
G = sm.symbols("G")
nu = sm.symbols("nu")
w_tilde = sm.symbols("w_tilde")
tau = sm.symbols("tau")
rho = sm.symbols("rho")
w_tilde = sm.symbols("w_tilde")
sigma = sm.symbols("sigma")
epsilon = sm.symbols("epsilon")

In [26]:
# 1. Define the utility function
utility = sm.log(C**alpha * G**(1-alpha)) - nu * L**2 / 2

# 2. Define the constraint equation
constraint = sm.Eq(C, kappa + (1 - tau) * w * L)

# 3. Substitute the value of w_tilde in the constraint equation
constraint_subs = constraint.subs(w, w_tilde / (1 - tau))

# 4. Isolate L from the budget constraint
C_from_con = sm.solve(constraint_subs, C)[0]

# 5. Substitute the expression for C in the utility function
utility_subs = utility.subs(C, C_from_con)

# 6. Find the first-order condition by differentiating the utility function with respect to L
foc = sm.diff(utility_subs, L)

# 7. Solve the first-order condition equation for L
sol = sm.solve(sm.Eq(foc, 0), L)
sol[0]

(-kappa*nu - sqrt(nu*(4*alpha*w_tilde**2 + kappa**2*nu)))/(2*nu*w_tilde)

This solution is the same as the solution as we are asked to verify in the question 

**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

In [ ]:
# 1. Setting the baseline parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * 16**2)
tau = 0.30

# 2. Defining the range of w values
w_values = np.linspace(0.1, 10, 100)

# 3. Computing the optimal labor supply
optimal_labor = ex.optimal_labor_supply(w_values, alpha, kappa, nu, tau)

# 4 Plotting the results
plt.plot(w_values, optimal_labor)
plt.xlabel('Wage, w')
plt.ylabel('Optimal Labor Supply')
plt.title('Optimal Labor Supply as a Function of w')
plt.show()

Here we can see that for a real wage between 0 and 2 the the labor supply increases substantially. From about a real wage of 2 and onwards there is marginal diminishing returns in labor supply and the optimal labor supply is about 15.8


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


In [ ]:
# 1. Setting the baseline parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * 16**2)
w = 1.0

# 2. Defining the grid of tau values
tau_values = np.linspace(0.1, 0.9, 100)

# 3. Computing the implied values for L, G, and utility for each tau value
L_values = []
G_values = []
utility_values = []

# 4. Computing the implied values for L, G, and utility for each tau value using loop
for tau in tau_values:
    # a. Computing the optimal labor supply, government consumption, and utility
    w_tilde = (1 - tau) * w
    L_opt = ex.optimal_labor_supply_w_tilde(w_tilde, alpha, kappa, nu)
    G = ex.government_consumption(tau, w, alpha, kappa, nu)
    utility = ex.worker_utility(tau, w, alpha, kappa, G, nu)
    
    # b. Appending the results to the lists
    L_values.append(L_opt)
    G_values.append(G)
    utility_values.append(utility)

# 5. Plotting the results
plt.figure(figsize=(12, 4))

# 6. Creating 3 subplots
plt.subplot(1, 3, 1)
plt.plot(tau_values, L_values)
plt.xlabel('Tau')
plt.ylabel('Labor Supply (L)')
plt.title('Labor Supply as a Function of Tau')

plt.subplot(1, 3, 2)
plt.plot(tau_values, G_values)
plt.xlabel('Tau')
plt.ylabel('Government Consumption (G)')
plt.title('Government Consumption as a Function of Tau')

plt.subplot(1, 3, 3)
plt.plot(tau_values, utility_values)
plt.xlabel('Tau')
plt.ylabel('Worker Utility')
plt.title('Worker Utility as a Function of Tau')

plt.tight_layout()
plt.show()

In the first plot we can see that the higher the tax-rate the lower the labor supply. 

In the second plot the Goverment consumption function is plottet for tau. Here we can see that the higher the tax the higher the consumption until it reaches 
the limit where there is no labor supply to pay taxes tau.

In the third plot the worker utility function of tau is plottet. It looks like a laffer curve where we can see that the optimal taxrate is approximately 50% 


**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

In [ ]:
# 1. Finding the index of the maximum utility value
max_utility_index = np.argmax(utility_values)

# 2. Getting the optimal tau and maximum worker utility
tau_optimal = tau_values[max_utility_index]
worker_utility_max = utility_values[max_utility_index]

# 3. Creating plot
plt.figure(figsize=(8, 4))
plt.subplot(1, 1, 1)

# 4. Plotting the results
plt.plot(tau_values, utility_values)
plt.axvline(x=tau_optimal, color='r', linestyle='--', label='Optimal Tau')

# 5. Adding labels and title
plt.xlabel('Tau')
plt.ylabel('Worker Utility')
plt.title('Worker Utility as a Function of Tau')
plt.legend()
plt.ylim(bottom=1.25)  
plt.show()

# 6. Printing the results
print(f'Optimal tau: {tau_optimal:.3f}')
print(f'Maximum worker utility: {worker_utility_max:.3f}')

As mentioned in the last question the optimal tax-rate looked like it was close to 50% and we can hereby confirm that the optimal taxrate that optimizes worker utility is 51,212 %

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{\sigma-1} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [ ]:
# 4. Set_1 parameters
sigma1 = 1.001
rho1 = 1.001
epsilon = 1.0
tau1 = tau_optimal # Using the tau we found in question 4 

# 5. Set_2 parameters
sigma2 = 1.5
rho2 = 1.5
tau2 = tau_optimal # Using the tau we found in question 4 

# 6. Finding G for Set 1
G_set1 = ex.find_G(w, tau1, alpha, kappa, sigma1, rho1, epsilon, nu)

# 7. Finding G for Set 2
G_set2 = ex.find_G(w, tau2, alpha, kappa, sigma2, rho2, epsilon, nu)

# 8. Printing the results: 
print("G for Set 1: {:.4f}".format(G_set1))
print("G for Set 2: {:.4f}".format(G_set2))

We can hereby conclude that from the optimal labour the optimal consumption of G is: 

for Set 1: 7.6765

for Set 2: 4.2077

**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In [ ]:
# 1. Find the socially optimal tax rate for Set 1
result_set1 = optimize.minimize_scalar(ex.objective_set1, bounds=(0, 1), method='bounded')

# 3. Using if statement as debugger to print solution if found
if result_set1.success:
    tau_star_set1 = result_set1.x
    print("Socially optimal tax rate for Set 1 is tau =", round(tau_star_set1, 4))
else:
    raise ValueError("Optimization failed for Set 1.")

# 3. Finding the socially optimal tax rate for Set 2
result_set2 = optimize.minimize_scalar(ex.objective_set2, bounds=(0, 1), method='bounded')

# 4. Using if statement as debugger to print solution if found
if result_set2.success:
    tau_star_set2 = result_set2.x
    print("Socially optimal tax rate for Set 2 is tau =", round(tau_star_set2, 4))
else:
    raise ValueError("Optimization failed for Set 2.")

Penalty term explanation: 

So in the code we use: "np.maximum(0, -constraint_val)" this calculates the maximum value between 0 and the negative value of our "constraint_val." 

The purpose of this is to ensure that if our "constraint_val" is negative (i.e., the constraint is violated), the penalty term will be positive, and if our "constraint_val" is non-negative (i.e., the constraint is satisfied), the penalty term will be zero.

Here we then insert 1e8 which is a constant scaling factor used to amplify the penalty. It multiplies the result of np.maximum(0, -constraint_val) to make the penalty term relatively large in comparison to other components of the utility funciton.

Hereby when we multiply the penalty term with a large value, the utility function is penalized heavily when the constraint is violated, effectively discouraging solutions that do not satisfy the constraint. This encourages the optimization algorithm to prioritize solutions that satisfy the constraint.

The penalty term is then used in the utilityfunctions objective_set1 and objective_set2 to adjust the utility value. The higher the penalty, the more the utility is reduced when the constraint is violated, leading to a lower overall utility value for such solutions and therefore we can find the optimal solution that does satisfy the constraint. 

From this method we came to the conclusion that the Socially optimal tax rate for Set_1 and Set_2 are the above floats. 

## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

To verify nummerically $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$. We calculate different values of profit for generated values of kappa and l where each are compared with our current highest max profit.

We then plot our optimal solution in a 3D plot.

In [ ]:
# 1. Calling class
p2 = problem2()

# 2. Calling function
p2.profit_max1()

#3. Printing results
print("Maximum profit:", p2.sol.max_profit)
print("Optimal l:", p2.sol.opt_l)
print("Optimal kappa:", p2.sol.opt_kappa)

# 3. Creating 3D plot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(p2.sol.l_grid, p2.sol.kappa_grid, p2.sol.profit_grid, cmap='plasma', alpha=0.8)

# 4. Add the optimal point to the plot
ax.scatter([p2.sol.opt_kappa], [p2.sol.opt_l], [p2.sol.max_profit], color='red', s=100, edgecolors='black', marker='o')
ax.text(p2.sol.opt_kappa, p2.sol.opt_l, p2.sol.max_profit, f"Optimal Point: ({p2.sol.opt_kappa:.2f}, {p2.sol.opt_l:.2f}, {p2.sol.max_profit:.2f})", color='black', fontsize=12)

# 5. Set labels and titles
ax.set_xlabel('l value')
ax.set_ylabel('kappa value')
ax.set_zlabel('Profit')
ax.set_title('Profit as a function of kappa and l')

plt.show()

The value for $\kappa\in\left\{1.0 , 2.0\right\}$ that maximises our profit is $\kappa=2.0$. This is the highest value in our chosen interval $\kappa$ symbolizes a demand shock a positive $\kappa$-value is a positive demand shock and implies we employ more hairdressers. Therefore it makes sense that the $\kappa$-value that maximizes profit is the highest possible.

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [ ]:
# 1. Calling class
p2 = problem2()

# 2. Calling function
p2.profit_max2()

We have choosen a value of $K=100000$ as this almost gives the same value of H each time the simulation is run.

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
# 1. Calling class
p2 = problem2()

# 2. Calling function
p2.profit_max3()

We see that if we follow the new policy where the value of $\Delta$ determines the amount of employees our profit is marginally higher.

**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
# 1. Calling class
p2 = problem2()

# 2. Calling function
p2.opt_delta()

# Notice: This might take a while to run

The optimal value of Delta is: $\Delta\approx0.076$ this varies a small amount with each simulation. 

The optimal value is higher than what we choose in question 3, this new value of $\Delta$ also gives a slightly higher profit. A higher value of $\Delta$ implies that we choose  $\ell_{t-1}$ more often and thereby stick to our previous number of employees lowering the cost associated with hiring and firing employees. This in turn results in a higher profit, H. 


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



As an alternative policy we have choosen to calculate an expected value of $\kappa$. By taking the average of the 5 previously estimated $\kappa$-values. $\kappa_{expected}$ is then inserted into our policy funtion:

$\ell_{t(new)}=\left(\frac{(1-\eta)(\alpha*\kappa_{t}+(1-\alpha)*\kappa_{t}^{expected})}{w}\right)^{\frac{1}{\eta}}$

Hereby our actor makes a decision on how many to employ based on the previous demand shocks.

In [ ]:
# 1. Calling class
p2 = problem2()

# 2. Calling function
p2.profit_max5()

We see that our profit is marginally lower than the one generated in question 3, implying that our policy change did not increase the profit. 

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

We start by defining the Griewank function, which is known for having many regularly distributed local minima. We have generated a Griewank plot below to show its destinctive shape.

It is firstly being defined as a wrapper function "griewank", which wraps around our "griewank_" function. 
We need the wrapper function when using our optimization algorithm (BFGS), as our wrapper function allows the Griewank fucntion to take a single argument, an array, instead of multiple arguments. 

In [ ]:
# 1. Defing wrapper function
def griewank(x):
 return griewank_(x[0],x[1])

# 2. Actual Griewank function
def griewank_(x):
    A = x[0]**2/4000 + x[1]**2/4000
    B = np.cos(x[0]) * np.cos(x[1]/np.sqrt(2))
    return A - B + 1

In [ ]:
# 1. Generating my x and y values
x = np.linspace(-10, 10, 500)
y = np.linspace(-10, 10, 500)

# 2. Creating a meshgrid of our x and y values
X, Y = np.meshgrid(x, y)

# 3. Use a list comprehension to evaluate the function at each point
Z = np.array([griewank_(np.array([x,y])) for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = Z.reshape(X.shape)

# 4. Create a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='magma')

# 5. Set labels and title
ax.set_title('Griewank function')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

Because we are using the BFGS optimizer we have to set warm up iterations where we use randomly generated initial guess for x.
After the warmup period our optimizer focus on a specific chosen point and minimizes this. 

We also set a tolerance which says that if the function value of a current solution comes close enough to the optimal solution, our algorithm stops eventhough it might not have found the true minimum.

In [ ]:
# 1. Setting parameters for BFGS optimizer
lower_bound = np.array([-600, -600]) 
upper_bound = np.array([600, 600])
K_warm = 10 # warmup iterations
K_max = 1000 # max iterations
tau = 1e-8 # tolerance
repeat = 10 # number of times to repeat the optimizer

In [ ]:
# 1. Run the repeated optimizer
x_opt, y_opt, x_initial_guesses, overall_iterations = ex.repeated_global_optimize(griewank_, lower_bound, upper_bound, K_warm, K_max, tau, repeat)

# 2. Print result
print("Optimal solution: ", x_opt)
print("Minimum function value: ", y_opt)
print("Total iterations: ", overall_iterations)

# 3. Plotting initial guesses
plt.figure(figsize=(10, 6))
x_initial_guesses = np.array(x_initial_guesses)
plt.scatter(range(len(x_initial_guesses)), x_initial_guesses[:, 0], label="x1", alpha=0.7)
plt.scatter(range(len(x_initial_guesses)), x_initial_guesses[:, 1], label="x2", alpha=0.7)

# 4. Setting label and legend
plt.xlabel("Iteration")
plt.ylabel("Initial guess value")
plt.title("Initial guesses for each iteration")
plt.legend()

plt.show()

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

During the warmup iterations our initial number is set at a randomly generated point and the optimization is performed independently from previous iterations. By increasing our number of iterations from 10 to 100 we achieve a more "rough" understanding of the dataset, we spend more time exploring the dataset. This reduces the time spend on optimizing for a single points. Therefore it is a trade-off. We are measuring the computation time for each of the two scenarios and the iterations used to find the optimal solution. 

The convergence rate might be faster with a higher number of warmup iterations, if one of the randomly generated points happen to be close to a minima, but we can not be sure of this. It might as well converge slower. 

Because we are working with a Griewank function which is known for having multiple local minima, it might be an idea to increase the number of warmup iterations in order to have a higher chance of exploring a global minimum.

In our solution above we ran the optimizer 10 times in order to increase our chances of exploring a global minimum. If the optimization code is only run once the chance of hitting a local minima that is not a global minimum was quite high. In the code below we will only be running the optimizer once with both 10 and 100 warmup iterations. 

In [ ]:
# 1. Choosing our two warmup counts
warmup_counts = [10, 100]

# 2. Creating loop to iterate over warmup counts and optimizing
for warmup_count in warmup_counts:

    # a. Start time measurement
    start_time = time.time()

    # b. Run the optimizer
    x_star, y_star, x_initial_guesses, iterations = ex.global_optimize(
        griewank_, lower_bound, upper_bound, warmup_count, K_max, tau)

    # c. End time measurement
    end_time = time.time()

    # d. Print the results
    print(f"Warmup count: {warmup_count}")
    print(f"Optimal solution: {x_star}")
    print(f"Minimum function value: {y_star}")
    print(f"Execution time: {end_time - start_time:.3f} seconds")
    print(f"Number of iterations: {iterations}")
    
    # e. Creating plot
    plt.figure(figsize=(10, 6))

    # f. Plotting values
    x_initial_guesses = np.array(x_initial_guesses)
    plt.scatter(range(len(x_initial_guesses)), x_initial_guesses[:, 0], label="x1", alpha=0.7)
    plt.scatter(range(len(x_initial_guesses)), x_initial_guesses[:, 1], label="x2", alpha=0.7)

    # g. Adding vertical line at warmup count
    plt.axvline(x=warmup_count, color='green', linestyle='--', label=f"Warmup iterations={warmup_count}")  # add vertical line at x=10
    
    # h. Setting labels and legend
    plt.xlabel("Iteration")
    plt.ylabel("Initial guess value")
    plt.title(f"Initial guesses for each iteration (Warmup count: {warmup_count})")
    plt.legend()
    
    plt.show()

The first plot is for 10 warmup iterations and the other for 100. We see a clear visible difference in our two plots, as we would expect the first 100 iterations are a lot less centered when 100 warmup iterations are chosen, as these are randomly uniform distributed. The end of the warmup is shown by the green vertical line.

We have measuered the computation time it has to find the minimum function value for both warmup choices, the execution time is always smaller when running the code with 10 warmups rather than 100. Although the time difference is quite small. 

As well as this we have measured the number of iterations it takes to find the optimal value for the different warmup choices. Whenever we run the code we see that the number of iterations is always lower for 10 warmups, often being around half the number of iterations it takes the one with 100 warmups. This is because when using 10 warmups it quickly finds a possible global minimum and "explores" this in-depth. 

An upside to choosing 100 iterations is that because the optimizer explores the dataset more, it always find the global minimum. Whereas our optimizer with 10 warmups sometimes find a local minima within our tolerance and stops there. 